In [1]:
from pywgraph import *

In [2]:
group = CommonGroups.RealMultiplicative
group.strict_total_order_function = lambda x, y: abs(x) < abs(y)

In [8]:
grafo = WeightedDirectedGraph.from_dict(
    {
        'a': {'b': 1, 'c': 6},
        'b': {'a': 2, 'd': 4},
        'c': {'a': -5},
        'd': {'b': 4},
    },
    group
)

In [9]:
caminos = grafo.find_paths(start='a', end='b', max_weight=500, general_max_visitations=5, max_iter=40)
[grafo.path_weight(path) for path in caminos]

[1.0, 2.0, 16.0, -30.0, 32.0, 2.0, -60.0, 32.0, 16.0, -30.0, -480.0, -60.0]

In [ ]:
def _iter_aux(
    grafo: WeightedDirectedGraph,
    explorer: PathExplorerPlus,
    target: str,
    general_max_visitations: int,
    specific_max_visitations: dict[str, int],
    max_weight
) -> tuple[list[Path], list[PathExplorerPlus]]:

    current_node = explorer.path[-1]
    weight = explorer.weight
    visitations = explorer.visitations.copy()
    visitations[current_node] = visitations.get(current_node, 0) + 1
    current_node_vistiations = visitations[current_node]
    current_node_max_vistiations = specific_max_visitations.get(
        current_node, general_max_visitations
    )
    found_path: list[Path] = []
    new_explorers: list[PathExplorerPlus] = []

    if current_node_vistiations > current_node_max_vistiations:
        return found_path, new_explorers
    
    if max_weight is not None: 
        if grafo.group.le(max_weight, weight):
            return found_path, new_explorers

    if current_node == target:
        found_path = [explorer.path]

    children_tuples: set[tuple[str, "Group.element"]] = grafo.children_with_weight(current_node)
    forbidden_nodes = {
        node
        for node, rep in visitations.items()
        if rep >= specific_max_visitations.get(node, general_max_visitations)
    }
    unexplored_nodes = {(child, child_weight) for child, child_weight in children_tuples if child not in forbidden_nodes}
    if not unexplored_nodes:
        return found_path, new_explorers

    new_explorers = [
        PathExplorerPlus(Path(explorer.path + [node]), grafo.group(weight, child_weight), visitations)
        for node, child_weight in unexplored_nodes
    ]
    return found_path, new_explorers

def _find_paths(
    grafo: WeightedDirectedGraph,
    start: str,
    end: str,
    general_max_visitations: int = 1,
    specific_max_visitations: dict[str, int] = {},
    max_iter: int = 1_000,
    max_paths: int | None = None,
    max_weight = None,
) -> list[Path]:
    if max_paths is None:
        m_paths: int | float = inf
    else:
        m_paths = max_paths
    explorers: list[PathExplorerPlus] = [PathExplorerPlus(Path([start]), grafo.group.identity)]
    all_paths: list[Path] = []

    it = 1
    while explorers and (it < max_iter) and (len(all_paths) < m_paths):
        discovered_path, discovered_explorers = _iter_aux(
            grafo,
            explorers.pop(0),
            end,
            general_max_visitations,
            specific_max_visitations,
            max_weight
        )
        all_paths.extend(discovered_path)
        new_explorers = list(set(discovered_explorers) - set(explorers))
        explorers.extend(new_explorers)

        it += 1

    if it == max_iter:
        warn(f"Max iterations reached ({max_iter})", Warning)

    return all_paths

In [ ]:
_find_paths(grafo, "a", "c", general_max_visitations=5)